In [1]:
import nltk
import json
import string
import numpy as np
import pandas as pd
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [2]:
import os

data_folder=os.path.join('data','texts_18-80')
all_files=[os.path.join(data_folder, fname) for fname in os.listdir(data_folder)]

train_6_9={}
test_6_9={}
count=0

for d in all_files:
    bname=os.path.basename(d)
    file_num=os.path.splitext(bname)[0]
    with open(d, 'r') as f:
        if count <=5:
            test_6_9[file_num]=f.read()
        elif count in range(5, 30):
            train_6_9[file_num]=f.read()
        count=count+1

In [3]:
def syllables(word):
    count = 0
    vowels = 'aeiou'    
    if word[0] in vowels:
        count +=1
    for index in range(1,len(word)):
        if word[index] in vowels and word[index-1] not in vowels:
            count +=1
        if word.endswith('e'):
            count -= 1
        if word.endswith('le'):
            count+=1
        if word.endswith('ie'):
            count+=1
        if count == 0:
            count +=1
    return count

In [4]:
def Readability(text):    
    cleaned_words=[[]]    
    for k, v in text.items():
        values=v
        key=k
        split_sentences=tokenizer.tokenize(str(values).replace("\\n"," "))
    tokens=[x.split() for x in split_sentences]
    for w in tokens:
        cleaned_words = [[''.join(c for c in s if c not in string.punctuation) for s in m] for m in tokens]
        cleaned_words = [[s.lower() for s in k if s] for k in cleaned_words]
        cleaned_words=[[w for w in k if w.isalpha()] for k in cleaned_words]
    words=[a for m in cleaned_words for a in m ]
    Total_sentences=len(cleaned_words)
    Total_words=len(words)
    Total_Syllable=0
    for w in words:
        Total_Syllable+=syllables(w)
    A=(Total_words)/(Total_sentences)
    B=(Total_Syllable)/(Total_words)
    flesch_kincaid_difficulty=(206.835-(1.015*A)-(84.6*B))
    flesch_kincaid_grade_level=(0.39*A)+(11.8*B)-15.59                 
    with open('data/readability.txt','a', encoding="utf-8") as f:
        print('{} {} {}'.format(key, round(flesch_kincaid_difficulty), round(flesch_kincaid_grade_level), 'label'), file=f)            

In [5]:
model1=pd.read_csv('data/readability_18_80_train.txt', names=['Title', 'Difficulty', 'Grade', 'Category'], delimiter='  ')
model1[:5]

/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


,Title,Difficulty,Grade,Category
0,Lake_Titicaca,61,9,18-80
1,Vincent_van_Gogh,61,10,18-80
2,Justin_Beiber,58,10,18-80
3,Rembrandt,50,13,18-80
4,Iliad,59,11,18-80


In [6]:
model2=pd.read_csv('data/readability_6_9_train.txt', names=['Title', 'Difficulty', 'Grade', 'Category'], delimiter=' ')
model2[:5]

,Title,Difficulty,Grade,Category
0,Elephants.pdf,71,7,6-9
1,grandmas-glasses-FKB-kids-stories.pdf,70,6,6-9
2,The-story-of-stories-FKB-kids-books.pdf,66,7,6-9
3,Free-Kids-Books-Spelling-Lists-workbook-3-comi...,37,21,6-9
4,Cats-FKB-Kids-Stories.pdf,70,6,6-9


In [8]:
model3=pd.read_csv('data/readability_10_13_train.txt', names=['Title', 'Difficulty', 'Grade', 'Category'], delimiter=' ')
model3[:5]

,Title,Difficulty,Grade,Category
0,The-Blob-The-Frog-The-DOG-and-The-Girl.pdf,88,4,10-13
1,Sugar-the-Robot-and-the-race-to-save-the-Earth...,85,4,10-13
2,Alice-in-Wonderland.pdf,87,6,10-13
3,gifted1.pdf,91,3,10-13
4,Beekle-Henry.pdf,74,7,10-13


In [9]:
def Sentences(text):    
    cleaned_words=[[]] #given the sentences, put the words in a multidimensional array    
    for k, v in text.items(): #iterating over a dictonary
        values=v
        key=k        
        for kk in key: #for each key, access the values "which are sentences in this case"
            split_sentences=tokenizer.tokenize(str(values).replace("\\n"," ")) #use nltk sentence tokenizer to split into sentences
        tokens=[x.split() for x in split_sentences] #get the unique words in each sentence
        cleaned_words = [[''.join(c for c in s if c not in string.punctuation) for s in m] for m in tokens] #remove punctuations
        cleaned_words = [[s.lower() for s in k if s] for k in cleaned_words] #lowercase the words
        cleaned_words=[[w for w in k if w.isalpha()] for k in cleaned_words] #ignore non-aplhabets      
        Total_sentences=[' '.join(c for c in s ) for s in cleaned_words]  #join them back to be words, so they don't appear as list items      
        for s in Total_sentences: #for each sentence identified with the key, write to the respective file
            with open('data/Sentences_'+key+'_.txt', 'a') as f: #i identified the files with the key
                print(s, file=f)        

In [10]:
dummy_docs={}
dummy_docs['frozen_disney']='Frozen is a 2013 American 3D computer-animated musical fantasy film produced by Walt Disney Animation Studios and released by Walt Disney Pictures.[4] It is the 53rd Disney animated feature film. Inspired by Hans Christian Andersens fairy tale The Snow Queen,[5] the film tells the story of a fearless princess who sets off on a journey alongside a rugged iceman, his loyal pet reindeer, and a naïve snowman to find her estranged sister, whose icy powers have inadvertently trapped the kingdom in eternal winter.'
dummy_docs['empire']='Empire is an American musical drama television series created by Lee Daniels and Danny Strong which debuted on January 7, 2015. Although it is filmed in Chicago,[2][3] the show is set in New York. It centers on a fictional hip hop music and entertainment company, Empire Entertainment, and the drama among the members of the founders family as they fight for control of it.'
dummy_docs['finding dory']='Finding Dory is a 2016 American 3D computer-animated comedy-drama adventure film produced by Pixar Animation Studios and released by Walt Disney Pictures. Directed by Andrew Stanton with co-direction by Angus MacLane,[5][6] the screenplay was written by Stanton and Victoria Strouse.[7] The film is a sequel/spinoff[8][9] to 2003s Finding Nemo and features the returning voices of Ellen DeGeneres and Albert Brooks, with Hayden Rolence (replacing Alexander Gould), Ed O Neill, Kaitlin Olson, Ty Burrell, Diane Keaton and Eugene Levy joining the cast. Finding Dory focuses on the amnesiac fish Dory, who journeys to be reunited with her parents.[10'

In [11]:
Sentences(dummy_docs)

In [12]:
Readability(dummy_docs)